In [11]:
import torch
import torch.nn as nn
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"
        self.d_out = d_out
        self.num_heads = num_heads

        #Step 2
        self.head_dim = d_out // num_heads 

        #Step 3
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out) 

        self.dropout = nn.Dropout(dropout)
        
        #...Step 8 
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )
    
    def forward(self, x):
        b, num_tokens, d_in = x.shape

        #Step 4
        keys = self.W_key(x) 
        queries = self.W_query(x) 
        values = self.W_value(x) 

        #Step 5
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim) 

        #Step 6
        keys = keys.transpose(1, 2) 
        queries = queries.transpose(1, 2) 
        values = values.transpose(1, 2) 

        #Step 7
        attn_scores = queries @ keys.transpose(2, 3) 

        #Step 8
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens] 
        attn_scores.masked_fill_(mask_bool, -torch.inf) 
        attn_weights = torch.softmax(
                        attn_scores / keys.shape[-1]**0.5, dim=-1
                    )
        attn_weights = self.dropout(attn_weights)

        #Step 9 + Step 10
        context_vec = (attn_weights @ values).transpose(1, 2)

        #Step 10 + Step 11
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)
        
        return context_vec

In [14]:
#Step 1
a = torch.tensor([[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],
                )

torch.manual_seed(123)
batch=torch.stack((a,a),dim=0)
print(batch.shape)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

torch.Size([2, 3, 4])
tensor([[[ 0.6326, -0.4115],
         [ 0.7489, -0.2632],
         [ 0.7444, -0.2735]],

        [[ 0.6326, -0.4115],
         [ 0.7489, -0.2632],
         [ 0.7444, -0.2735]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 3, 2])
